Notes in progress ...

<dl>
    <dt><b>Aim</b></dt>
  <dd>Determine the degree of dispersion, per day, amongst a state's delta curves.</dd>
  <dt><b>Why?</b></dt>
    <dd>In the case of <i>positive rate delta curves</i>, as the dispersion increases from zero, the more likely an impending outbreak [mathematical proof].  In the case of <i>hospitalization rate delta curves</i>, as the dispersion increases from zero, it is quite probable that hospitalisations will increase rapidly [contingency planning alert?] </dd>
</dl>

<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import numpy as np

import logging

import os
import pathlib
import sys


<br>

### Paths

In [2]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [3]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'dispersions')

<br>

Appending Paths

In [4]:
sys.path.append(parent)

<br>

### Logging

In [5]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
logger.info(root)

INFO:__main__:J:\library\projects\sars\fundamentals\atlantic\notebooks\warehouse


In [7]:
logger.info(warehouse)

INFO:__main__:J:\library\projects\sars\fundamentals\atlantic\notebooks\warehouse\dispersions


<br>

### Custom

In [8]:
import atlantic.base.directories

<br>

Set-up directories

In [9]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

## Data

In [10]:
datauri = os.path.join(root, 'trends', 'percentages.csv')

parse_dates = ['datetimeobject']
percentages = pd.read_csv(filepath_or_buffer=datauri, header=0, encoding='utf-8', parse_dates=parse_dates)

In [11]:
logger.info(percentages.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250120 entries, 0 to 250119
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   datetimeobject                  250120 non-null  datetime64[ns]
 1   STUSPS                          250120 non-null  object        
 2   period                          250120 non-null  object        
 3   deathRateDelta                  250120 non-null  float64       
 4   deathRate                       250120 non-null  float64       
 5   positiveRateDelta               250120 non-null  float64       
 6   positiveRate                    250120 non-null  float64       
 7   testRateDelta                   250120 non-null  float64       
 8   testRate                        250120 non-null  float64       
 9   icuRateDelta                    250120 non-null  float64       
 10  icuRate                         250120 non-null  float64

<br>
<br>

## Calculations

In [12]:
sections = ['positiveRateDelta', 'deathRateDelta', 'hospitalizedRateDelta']

<br>

### Functions

<br>

Baseline table

In [13]:
def baseline(data: pd.DataFrame, section: str):
    
    structure = pd.pivot_table(data, index=['datetimeobject', 'STUSPS'], columns=['period'], values=[section])
    structure.dropna(axis=0, how='any', inplace=True)

    periodfields = structure.columns

    structure.loc[:, 'range'] = structure[periodfields].max(axis=1) - structure[periodfields].min(axis=1)
    structure.loc[:, 'midpoint'] = 0.5*structure['range'] + structure[periodfields].min(axis=1)
    structure.loc[:, 'median'] = structure[periodfields].median(axis=1)

    structure.reset_index(drop=False, inplace=True)
    matrix = structure[['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median']].values
    
    return pd.DataFrame(data=matrix, columns=['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median'])


<br>

Scores

In [14]:
def scores(data: pd.DataFrame):
    
    blob = data.copy()
    
    scores = blob[['range', 'midpoint']].apply(lambda x: x['midpoint'] * np.log(x['range']) if x['range'] > 0 else 0, axis=1)
    
    return pd.concat([blob, scores.rename('score')], axis=1)


<br>

Latest

In [15]:
def latest(data: pd.DataFrame):
    
    blob = data.copy()
    
    condition = blob['datetimeobject'] == blob['datetimeobject'].max()
    
    return blob[condition].sort_values(by='rank')
    

<br>

### Divergence

In [16]:
for section in sections:
    
    # A data set w.r.t. a measure
    example = percentages[['datetimeobject', 'STUSPS', 'period', section]]
    logger.info('\n{}\n'.format(section))
    
    # ['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median'] 
    data = baseline(data=example, section=section)
        
    # ['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median', 'score']
    data = scores(data=data)
            
    # ['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median', 'score', 'rank']
    ranks = data[['datetimeobject', 'score']].groupby(by='datetimeobject').rank(method='min', ascending=False).score
    data = pd.concat([data, ranks.rename('rank')], axis=1)    
    data.to_csv(path_or_buf=os.path.join(warehouse, section + 'Dispersion.csv'), header=True, encoding='utf-8', index=False)
    logger.info('\n{}\n'.format(data.head()))
    
    # ['datetimeobject', 'STUSPS', 'range', 'midpoint', 'median', 'score', 'rank']
    inbrief = latest(data=data)
    inbrief.to_csv(path_or_buf=os.path.join(warehouse, section + 'DispersionLatest.csv'), header=True, encoding='utf-8', index=False)
    logger.info('\n{}\n'.format(inbrief.head()))
    

INFO:__main__:
positiveRateDelta

INFO:__main__:
  datetimeobject STUSPS    range midpoint   median          score  rank
0     2020-03-31     AK  4166.67  2083.33  56.8162   17364.315905  50.0
1     2020-03-31     AL    98000    49000  301.461  563143.415125   6.0
2     2020-03-31     AR  5711.11  2855.56  113.347   24701.037785  44.0
3     2020-03-31     AZ    32000    16000  329.309  165975.858909  21.0
4     2020-03-31     CA  17034.9  8517.44   312.86   82985.593655  32.0

INFO:__main__:
      datetimeobject STUSPS    range midpoint   median      score  rank
17259     2021-02-25     VT  20.3683  10.1841  7.73144  30.694757   1.0
17253     2021-02-25     SC  13.4769  6.73844  4.88722  17.526509   2.0
17246     2021-02-25     NY  12.0858  6.04291  4.65324  15.059142   3.0
17242     2021-02-25     NH  11.7616  5.88081  4.69014  14.495266   4.0
17258     2021-02-25     VA  11.2291  5.61455  3.53674  13.578833   5.0

INFO:__main__:
deathRateDelta

INFO:__main__:
  datetimeobject STUSPS 

<br>
<br>

## End

In [17]:
%%bash

date +"%Y-%m-%d %T"

2021-02-26 23:15:07
